---
# **Instalación del entorno de Kaggle**



In [ ]:
!pip install -q kaggle

---
# **Instalación de librerias**

In [ ]:
import pandas as pd
import statistics
from sklearn.preprocessing import MinMaxScaler

---
# **Lectura de los datos desde Kaggle**

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!echo '{"username":"nicolsniovalderrama","key":"f7f9fda0ddecb368ca50593318da1412"}' > ~/.kaggle/kaggle.json

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c udea-ai4eng-20241

 76% 16.0M/21.2M [00:00<00:00, 161MB/s]
100% 21.2M/21.2M [00:00<00:00, 175MB/s]


In [ ]:
!unzip udea-ai4eng-20241

Archive:  udea-ai4eng-20241.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
df_train = pd.read_csv('train.csv')
df_train.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo


---
# **Tratamiento de datos**

## **Valores duplicados**

In [ ]:
df_train.duplicated().sum()

0

### Eliminación de columnas repetidas

In [ ]:
df0 = df_train.copy()
A = df0['FAMI_TIENEINTERNET'] == df0['FAMI_TIENEINTERNET.1']
print(f'Hay {len(A)} datos')
B = (A==False).sum()
print(f'De los cuales, {B} son diferentes entre las columnas FAMI_INTERNET Y FAMI_INTERNET.1')
C = B / len(A)
print(f'Los datos diferentes entre las dos columnas corresponde al {(C*100).round(2)}% deL total de los datos')

Hay 692500 datos
De los cuales, 26629 son diferentes entre las columnas FAMI_INTERNET Y FAMI_INTERNET.1
Los datos diferentes entre las dos columnas corresponde al 3.85% deL total de los datos


Debido a que las dos columnas difieren en solo 3.85%, se toma la desición de eliminar la columna 'FAMI_TIENEINTERNET.1'

In [ ]:
df1 = df0.drop('FAMI_TIENEINTERNET.1', axis = 1)

## **Información no relevante**

Se eliminan las variables ID, PERIODO, ESTU_PRIVADO_LIBERTAD, FAMI_TIENELAVADORA y FAMI_TIENEAUTOMOVIL ya que se considera que no aportaran al modelo.

In [ ]:
df1 = df1.drop(columns = ['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
                          'ESTU_PRIVADO_LIBERTAD', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL'], axis = 1)

## **Conversión de variables categoricas en numericas**

### **Transformación de RENDIMIENTO_GLOBAL**

In [ ]:
df1['RENDIMIENTO_GLOBAL'].unique()

array(['medio-alto', 'bajo', 'alto', 'medio-bajo'], dtype=object)

In [ ]:
df1['RENDIMIENTO_GLOBAL'] = df1['RENDIMIENTO_GLOBAL'].replace({'bajo': 1,
                                                               'medio-bajo': 2,
                                                               'medio-alto': 3,
                                                               'alto': 4})

### **Transformación de ESTU_VALORMATRICULAUNIVERSIDAD**

In [ ]:
df1['ESTU_VALORMATRICULAUNIVERSIDAD'].unique()

array(['Entre 5.5 millones y menos de 7 millones',
       'Entre 2.5 millones y menos de 4 millones',
       'Entre 4 millones y menos de 5.5 millones', 'Más de 7 millones',
       'Entre 1 millón y menos de 2.5 millones',
       'Entre 500 mil y menos de 1 millón', 'Menos de 500 mil',
       'No pagó matrícula', nan], dtype=object)

In [ ]:
df1['ESTU_VALORMATRICULAUNIVERSIDAD'] = df1['ESTU_VALORMATRICULAUNIVERSIDAD'].replace({'No pagó matrícula': 0,
                                                                                       'Menos de 500 mil': 250000,
                                                                                       'Entre 500 mil y menos de 1 millón': 750000,
                                                                                       'Entre 1 millón y menos de 2.5 millones': 1750000,
                                                                                       'Entre 2.5 millones y menos de 4 millones': 3250000,
                                                                                       'Entre 4 millones y menos de 5.5 millones': 4750000,
                                                                                       'Entre 5.5 millones y menos de 7 millones': 6250000,
                                                                                       'Más de 7 millones': 7000000
                                                                                       })

### **Transformación de FAMI_ESTRATOVIVIENDA**

In [ ]:
df1['FAMI_ESTRATOVIVIENDA'].unique()

array(['Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 2', 'Estrato 1',
       nan, 'Estrato 6', 'Sin Estrato'], dtype=object)

In [ ]:
df1['FAMI_ESTRATOVIVIENDA'] = df1['FAMI_ESTRATOVIVIENDA'].replace('Estrato ', '', regex=True)
df1['FAMI_ESTRATOVIVIENDA'].unique()

array(['3', '4', '5', '2', '1', nan, '6', 'Sin Estrato'], dtype=object)

In [ ]:
df1= df1[df1['FAMI_ESTRATOVIVIENDA'] != 'Sin Estrato']

### **Tranformación de ESTU_HORASSEMANATRABAJA**

In [ ]:
df1['ESTU_HORASSEMANATRABAJA'].unique()

array(['Menos de 10 horas', '0', 'Más de 30 horas', 'Entre 21 y 30 horas',
       'Entre 11 y 20 horas', nan], dtype=object)

In [ ]:
df1['ESTU_HORASSEMANATRABAJA'] = df1['ESTU_HORASSEMANATRABAJA'].replace({'Menos de 10 horas': 5,
                                                                         'Entre 11 y 20 horas': 15.5,
                                                                         'Entre 21 y 30 horas': 25.5,
                                                                         'Más de 30 horas': 30})

### **Transformación de FAMI_EDUCACIONPADRE**

In [ ]:
df1['FAMI_EDUCACIONPADRE'].unique()

array(['Técnica o tecnológica incompleta',
       'Técnica o tecnológica completa',
       'Secundaria (Bachillerato) completa', 'No sabe',
       'Primaria completa', 'Educación profesional completa',
       'Educación profesional incompleta', 'Primaria incompleta',
       'Postgrado', nan, 'Secundaria (Bachillerato) incompleta',
       'Ninguno', 'No Aplica'], dtype=object)

In [ ]:
df1['FAMI_EDUCACIONPADRE'] = df1['FAMI_EDUCACIONPADRE'].replace({'No sabe': 0, 'Ninguno': 0, 'No Aplica': 0,
                                                                 'Primaria incompleta': 1, 'Primaria completa': 2,
                                                                 'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4,
                                                                 'Técnica o tecnológica incompleta': 5, 'Técnica o tecnológica completa': 6,
                                                                 'Educación profesional incompleta': 7, 'Educación profesional completa': 8,
                                                                 'Postgrado': 9})

### **Transformación de FAMI_EDUCACIONMADRE**

In [ ]:
df1['FAMI_EDUCACIONMADRE'].unique()

array(['Postgrado', 'Técnica o tecnológica incompleta',
       'Secundaria (Bachillerato) completa', 'Primaria completa',
       'Técnica o tecnológica completa',
       'Secundaria (Bachillerato) incompleta',
       'Educación profesional incompleta',
       'Educación profesional completa', 'Primaria incompleta', nan,
       'Ninguno', 'No sabe', 'No Aplica'], dtype=object)

In [ ]:
df1['FAMI_EDUCACIONMADRE'] = df1['FAMI_EDUCACIONMADRE'].replace({'No sabe': 0, 'Ninguno': 0, 'No Aplica': 0,
                                                                 'Primaria incompleta': 1, 'Primaria completa': 2,
                                                                 'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4,
                                                                 'Técnica o tecnológica incompleta': 5, 'Técnica o tecnológica completa': 6,
                                                                 'Educación profesional incompleta': 7, 'Educación profesional completa': 8,
                                                                 'Postgrado': 9})

## **Codificación binaria**

In [ ]:
var_yesno = ['FAMI_TIENEINTERNET', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR']

for var in var_yesno:
    df1.loc[df1[var] == 'Si', var] = 1
    df1.loc[df1[var] == 'No', var] = 0

In [ ]:
df1.head()

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,6250000.0,5,3,1,5.0,0,1,9.0,3
1,3250000.0,0,3,0,6.0,0,1,5.0,1
2,3250000.0,30,3,1,4.0,0,0,4.0,1
3,4750000.0,0,4,1,0.0,0,1,4.0,4
4,3250000.0,25.5,3,1,2.0,0,1,2.0,2


## **Valores nulos**


In [ ]:
df1.isnull().sum()

ESTU_VALORMATRICULAUNIVERSIDAD     6124
ESTU_HORASSEMANATRABAJA           30777
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26559
FAMI_EDUCACIONPADRE               23148
ESTU_PAGOMATRICULAPROPIO           6335
FAMI_TIENECOMPUTADOR              37963
FAMI_EDUCACIONMADRE               23622
RENDIMIENTO_GLOBAL                    0
dtype: int64

In [ ]:
df2 = df1.dropna()

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 632849 entries, 0 to 692499
Data columns (total 9 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ESTU_VALORMATRICULAUNIVERSIDAD  632849 non-null  float64
 1   ESTU_HORASSEMANATRABAJA         632849 non-null  object 
 2   FAMI_ESTRATOVIVIENDA            632849 non-null  object 
 3   FAMI_TIENEINTERNET              632849 non-null  object 
 4   FAMI_EDUCACIONPADRE             632849 non-null  float64
 5   ESTU_PAGOMATRICULAPROPIO        632849 non-null  object 
 6   FAMI_TIENECOMPUTADOR            632849 non-null  object 
 7   FAMI_EDUCACIONMADRE             632849 non-null  float64
 8   RENDIMIENTO_GLOBAL              632849 non-null  int64  
dtypes: float64(3), int64(1), object(5)
memory usage: 48.3+ MB


## **Estandarización de la variable ESTU_VALORMATRICULAUNIVERSIDAD**



In [ ]:
# Crear un objeto MinMaxScaler
scaler = MinMaxScaler()
# Ajustar y transformar la columna 'ESTU_VALORMATRICULAUNIVERSIDAD' usando .loc
df2.loc[:, 'ESTU_VALORMATRICULAUNIVERSIDAD'] = scaler.fit_transform(df2[['ESTU_VALORMATRICULAUNIVERSIDAD']])
# Redondear los valores a tres decimales usando .loc
df2.loc[:, 'ESTU_VALORMATRICULAUNIVERSIDAD'] = df2['ESTU_VALORMATRICULAUNIVERSIDAD'].round(3)

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 632849 entries, 0 to 692499
Data columns (total 9 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ESTU_VALORMATRICULAUNIVERSIDAD  632849 non-null  float64
 1   ESTU_HORASSEMANATRABAJA         632849 non-null  object 
 2   FAMI_ESTRATOVIVIENDA            632849 non-null  object 
 3   FAMI_TIENEINTERNET              632849 non-null  object 
 4   FAMI_EDUCACIONPADRE             632849 non-null  float64
 5   ESTU_PAGOMATRICULAPROPIO        632849 non-null  object 
 6   FAMI_TIENECOMPUTADOR            632849 non-null  object 
 7   FAMI_EDUCACIONMADRE             632849 non-null  float64
 8   RENDIMIENTO_GLOBAL              632849 non-null  int64  
dtypes: float64(3), int64(1), object(5)
memory usage: 48.3+ MB


In [ ]:
df2.head()

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,0.893,5,3,1,5.0,0,1,9.0,3
1,0.464,0,3,0,6.0,0,1,5.0,1
2,0.464,30,3,1,4.0,0,0,4.0,1
3,0.679,0,4,1,0.0,0,1,4.0,4
4,0.464,25.5,3,1,2.0,0,1,2.0,2


## **Estandarización de la variable ESTU_HORASSEMANATRABAJA**


In [ ]:
# Crear un objeto MinMaxScaler
scaler = MinMaxScaler()
# Ajustar y transformar la columna 'ESTU_VALORMATRICULAUNIVERSIDAD' usando .loc
df2.loc[:, 'ESTU_HORASSEMANATRABAJA'] = scaler.fit_transform(df2[['ESTU_HORASSEMANATRABAJA']])
df2['ESTU_HORASSEMANATRABAJA'] = df2['ESTU_HORASSEMANATRABAJA'].astype(float)
df2.loc[:, 'ESTU_HORASSEMANATRABAJA'] = df2['ESTU_HORASSEMANATRABAJA'].round(3)

<ipython-input-34-a16b453a5482>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ESTU_HORASSEMANATRABAJA'] = df2['ESTU_HORASSEMANATRABAJA'].astype(float)


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 632849 entries, 0 to 692499
Data columns (total 9 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ESTU_VALORMATRICULAUNIVERSIDAD  632849 non-null  float64
 1   ESTU_HORASSEMANATRABAJA         632849 non-null  float64
 2   FAMI_ESTRATOVIVIENDA            632849 non-null  object 
 3   FAMI_TIENEINTERNET              632849 non-null  object 
 4   FAMI_EDUCACIONPADRE             632849 non-null  float64
 5   ESTU_PAGOMATRICULAPROPIO        632849 non-null  object 
 6   FAMI_TIENECOMPUTADOR            632849 non-null  object 
 7   FAMI_EDUCACIONMADRE             632849 non-null  float64
 8   RENDIMIENTO_GLOBAL              632849 non-null  int64  
dtypes: float64(4), int64(1), object(4)
memory usage: 48.3+ MB


In [ ]:
df2.head()

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,0.893,0.167,3,1,5.0,0,1,9.0,3
1,0.464,0.000,3,0,6.0,0,1,5.0,1
2,0.464,1.000,3,1,4.0,0,0,4.0,1
3,0.679,0.000,4,1,0.0,0,1,4.0,4
4,0.464,0.850,3,1,2.0,0,1,2.0,2


## **Estandarización de la variable RENDIMIENTO_GLOBAL**

In [ ]:
# Crear un objeto MinMaxScaler
scaler = MinMaxScaler()
# Ajustar y transformar la columna 'ESTU_VALORMATRICULAUNIVERSIDAD' usando .loc
df2.loc[:, 'RENDIMIENTO_GLOBAL'] = scaler.fit_transform(df2[['RENDIMIENTO_GLOBAL']])
df2['RENDIMIENTO_GLOBAL'] = df2['RENDIMIENTO_GLOBAL'].astype(float)
df2.loc[:, 'RENDIMIENTO_GLOBAL'] = df2['RENDIMIENTO_GLOBAL'].round(3)

<ipython-input-37-1c48f0157eda>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['RENDIMIENTO_GLOBAL'] = df2['RENDIMIENTO_GLOBAL'].astype(float)


In [ ]:
df2['FAMI_ESTRATOVIVIENDA'].value_counts()

FAMI_ESTRATOVIVIENDA
2    224638
3    203434
1    107152
4     63045
5     22601
6     11979
Name: count, dtype: int64

In [ ]:
df2['FAMI_ESTRATOVIVIENDA'] = df2['FAMI_ESTRATOVIVIENDA'].astype(int)
df2['ESTU_HORASSEMANATRABAJA'] = df2['ESTU_HORASSEMANATRABAJA'].astype(int)
df2['FAMI_TIENEINTERNET'] = df2['FAMI_TIENEINTERNET'].astype(int)
df2['FAMI_TIENECOMPUTADOR'] = df2['FAMI_TIENECOMPUTADOR'].astype(int)
df2['ESTU_PAGOMATRICULAPROPIO'] = df2['ESTU_PAGOMATRICULAPROPIO'].astype(int)

<ipython-input-39-66326ce2f728>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['FAMI_ESTRATOVIVIENDA'] = df2['FAMI_ESTRATOVIVIENDA'].astype(int)
<ipython-input-39-66326ce2f728>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ESTU_HORASSEMANATRABAJA'] = df2['ESTU_HORASSEMANATRABAJA'].astype(int)
<ipython-input-39-66326ce2f728>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 632849 entries, 0 to 692499
Data columns (total 9 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ESTU_VALORMATRICULAUNIVERSIDAD  632849 non-null  float64
 1   ESTU_HORASSEMANATRABAJA         632849 non-null  int64  
 2   FAMI_ESTRATOVIVIENDA            632849 non-null  int64  
 3   FAMI_TIENEINTERNET              632849 non-null  int64  
 4   FAMI_EDUCACIONPADRE             632849 non-null  float64
 5   ESTU_PAGOMATRICULAPROPIO        632849 non-null  int64  
 6   FAMI_TIENECOMPUTADOR            632849 non-null  int64  
 7   FAMI_EDUCACIONMADRE             632849 non-null  float64
 8   RENDIMIENTO_GLOBAL              632849 non-null  float64
dtypes: float64(4), int64(5)
memory usage: 48.3 MB


In [ ]:
df2.head()

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,0.893,0,3,1,5.0,0,1,9.0,0.667
1,0.464,0,3,0,6.0,0,1,5.0,0.000
2,0.464,1,3,1,4.0,0,0,4.0,0.000
3,0.679,0,4,1,0.0,0,1,4.0,1.000
4,0.464,0,3,1,2.0,0,1,2.0,0.333
